# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import time

BATCH_SIZE = 16
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU(inplace=True)

        self.fc = nn.Linear(in_features=BATCH_SIZE*12*14*14, out_features=BATCH_SIZE*10)

    def forward(self, input):
        output = self.conv1(input)
        output = self.relu1(output)
        output = self.pool(output)
        output = self.conv2(output)
        output = self.relu2(output)
        output = output.view(1,-1 )
        output = self.fc(output)

        return output
torch.backends.cudnn.benchmark = True 
from torch.optim import Adam    
model = SimpleNet()
if torch.cuda.is_available():
    model.cuda()
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

625.0

Next, we can start to train and evaluate!

In [14]:
# evaluate
    # TODO:calculate the accuracy using traning and testing dataset
def evaluate(data_loader):
    model.eval()
    acc = 0
    for data in data_loader:
        img, label = data
        if torch.cuda.is_available():
            img = Variable(img).view(BATCH_SIZE,1,28, 28).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img).view(BATCH_SIZE,1,28, 28)
            label = Variable(label)
        out = model(img)
        out_reshape=out[0].view(BATCH_SIZE,10)
        _, pred = torch.max(out_reshape, 1)
        num_correct = (pred == label).sum()
        acc += num_correct.item()
    print('Accuracy:{:.6f}'.format(acc / (len(data_loader)*BATCH_SIZE)))
    
    
    

In [15]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    print(epoch)
    for images, labels in iter(train_loader):
        # TODO:forward + backward + optimize
        if torch.cuda.is_available():
            img = Variable(images.view(BATCH_SIZE,1,28, 28)).cuda()
            label = Variable(labels).cuda()
        else:
            img = Variable(images.view(BATCH_SIZE,1,28, 28))
            print("GPU not available")
            label = Variable(labels)
        out = model(img)
        out_reshape=out[0].view(BATCH_SIZE,10)
        loss = criterion(out_reshape, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        torch.cuda.empty_cache()
    evaluate(test_loader)      
        
        
        
        
    


0
Accuracy:0.907500
1
Accuracy:0.932800
2
Accuracy:0.928900
3
Accuracy:0.932000
4
Accuracy:0.936000
5
Accuracy:0.936900
6
Accuracy:0.940200
7
Accuracy:0.942100
8
Accuracy:0.938900
9
Accuracy:0.942300


#### Q5:
Please print the training and testing accuracy.

In [16]:
evaluate(train_loader)

Accuracy:0.939383
